<img src="embed.gif">

In [ ]:
import json
import math
import numpy as np
import openai
import os
import pandas as pd
import pytz
import requests
import sys

from datetime import datetime
from dotenv import load_dotenv
from IPython.display import display
from itertools import combinations
from PIL import Image
import emoji

In [ ]:
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version = os.getenv("AZURE_OPENAI_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")

# use an embeddingsmodel to create embeddings
def openai_text_embeddings(text, model=embedding_model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def text_comparison(vector1, vector2):
    """
    Cosine similarity value between two embedded vectors
    """
    if len(vector1) != len(vector2):
        print("[Error] Vectors do not have the same size")
        return None

    dot_product = sum(x * y for x, y in zip(vector1, vector2))
    magnitude1 = math.sqrt(sum(x * x for x in vector1))
    magnitude2 = math.sqrt(sum(x * x for x in vector2))
    cosine_similarity = round(dot_product / (magnitude1 * magnitude2), 15)

    # Or we can use directly the cosine_similarity function from Open AI

    if cosine_similarity == 1:
        decision = "identical"
        color_code = "\033[1;31;34m"
        emoticon = emoji.emojize(":red_heart:")

    elif cosine_similarity >= 0.8:
        decision = "similar semantic"
        color_code = "\033[1;31;32m"
        emoticon = emoji.emojize(":thumbs_up:")

    else:
        decision = "different"
        color_code = "\033[1;31;91m"
        emoticon = emoji.emojize(":fire:")

    print(
        f"{emoticon} {color_code}{decision.upper()} text (cosine similarity = {cosine_similarity})"
    )
    print("\033[0m")

# Text embeddings with Azure OpenAI

In [ ]:
text = "Hello. Good morning!"
emb = openai_text_embeddings(text)

In [ ]:
print("Text vector embedding size =", len(emb))
emb[:10]

In [ ]:
text_list = [
    "Appollo 11 mission",
    "A spaceship exploring the galaxy",
    "A muddy playground near the ocean",
    "A kid playing in the sand on the beach",
]

text_list

In [ ]:
combs = list(combinations(text_list, 2))

for idx, (text1, text2) in enumerate(combs, start=1):
    print(f"{idx} Comparing", text1, "vs", text2)
    emb1 = openai_text_embeddings(text1)
    emb2 = openai_text_embeddings(text2)
    text_comparison(emb1, emb2)

# Image embeddings with Azure Computer vision

In [ ]:
import os
import emoji
import json
from dotenv import load_dotenv
load_dotenv()
import matplotlib.pyplot as plt
import requests
from PIL import Image
import math

# Get Azure Computer Vision credentials
azure_cv_endpoint = os.getenv("azure_cv_endpoint")
azure_cv_key = os.getenv("azure_cv_key")

version = "?api-version=2023-02-01-preview&modelVersion=latest"

vec_img_url = (
    azure_cv_endpoint + "/computervision/retrieval:vectorizeImage" + version
)  # For doing the image vectorization

vec_txt_url = (
    azure_cv_endpoint + "/computervision/retrieval:vectorizeText" + version
)  # For the prompt vectorization

headers = {
    "Content-type": "application/json",
    "Ocp-Apim-Subscription-Key": azure_cv_key,
}

In [ ]:
def image_embedding(imageurl):
    """
    Embedding image using Azure Computer Vision 4
    """
    image = {"url": imageurl}
    r = requests.post(vec_img_url, data=json.dumps(image), headers=headers)
    image_emb = r.json()["vector"]

    return image_emb

In [ ]:
def images_comparison(vector1, vector2):
    """
    Get cosine similarity value between two embedded vectors
    """
    dot_product = sum(x * y for x, y in zip(vector1, vector2))
    magnitude1 = math.sqrt(sum(x * x for x in vector1))
    magnitude2 = math.sqrt(sum(x * x for x in vector2))
    cos_similarity = round(dot_product / (magnitude1 * magnitude2), 10)

    if cos_similarity == 1:
        decision = "identical"
        color_code = "\033[1;31;34m"
        emoticon = emoji.emojize(":red_heart:")
    elif cos_similarity >= 0.8:
        decision = "similar"
        color_code = "\033[1;31;32m"
        emoticon = emoji.emojize(":thumbs_up:")
    else:
        decision = "different"
        color_code = "\033[1;31;91m"
        emoticon = emoji.emojize(":fire:")

    print(
        f"{emoticon} {color_code}{decision.upper()} images (cosine similarity = {cos_similarity})"
    )

In [ ]:
url1 = os.getenv("IMAGE_PATH")+ "i4.jpg?raw=true"
car1_emb = image_embedding(url1)

plt.imshow(Image.open(requests.get(url1, stream=True).raw))
plt.axis("off")
plt.show()

In [ ]:
url2 = os.getenv("IMAGE_PATH")+ "i4_2.jpg?raw=true"
car2_emb = image_embedding(url2)

plt.imshow(Image.open(requests.get(url2, stream=True).raw))
plt.axis("off")
plt.show()

In [ ]:
url3 = os.getenv("IMAGE_PATH")+ "cat.jpg?raw=true"
cat_emb = image_embedding(url3)

plt.imshow(Image.open(requests.get(url3, stream=True).raw))
plt.axis("off")
plt.show()

In [ ]:
url4 = os.getenv("IMAGE_PATH")+ "car.jpg?raw=true"
something = image_embedding(url4)

plt.imshow(Image.open(requests.get(url4, stream=True).raw))
plt.axis("off")
plt.show()

In [ ]:
images_comparison(car1_emb, car1_emb)

In [ ]:
images_comparison(car1_emb, car2_emb)

In [ ]:
images_comparison(car1_emb, cat_emb)

In [ ]:
images_comparison(something, cat_emb)
images_comparison(something, car2_emb)
images_comparison(something, car1_emb)

In [ ]:
url5 = "https://f1.imgci.com/PICTURES/CMS/17700/17793.jpg"
something2 = image_embedding(url5)

plt.imshow(Image.open(requests.get(url5, stream=True).raw))
plt.axis("off")
plt.show()

url6 = "https://www.shell.com.tr/motorists/motorsports/motorsport/_jcr_content/pagePromo/image.img.960.jpeg/1466016547071/ferrari-f1-car-crossing-the-finishing-line.jpeg"
something3 = image_embedding(url6)

plt.imshow(Image.open(requests.get(url6, stream=True).raw))
plt.axis("off")
plt.show()

images_comparison(something2, something3)